# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [46]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import time
from sklearn.metrics import accuracy_score

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore). Don't forget to enrich the table with the 'dayofweek' column from the previous day's .csv-file.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [3]:
df = pd.read_csv("../data/day-of-week-not-scaled.csv")
df['dayofweek'] = pd.read_csv("../data/dayofweek.csv")['dayofweek']
df


,numTrials,hour,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,uid_user_15,...,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1,dayofweek
0,1,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
1,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
2,3,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
3,4,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
4,5,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1682,6,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1683,7,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3
1684,8,20,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3


In [4]:
Y = df['dayofweek']
X = df.drop(columns='dayofweek')

In [5]:
X_train,x_test,Y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=21,stratify=Y)

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [6]:
param_grid_svm = {
    'kernel' : ['linear','rbf','sigmoid'],
    'C' : [0.01,0.1,1,1.5,5,10],
    'gamma' : ['scale','auto'],
    'class_weight' : ['balanced','None']
}

In [7]:
model_SVM = SVC(random_state=21,probability=True)

In [8]:
grid_svm = GridSearchCV(model_SVM,param_grid=param_grid_svm,cv=5,scoring='accuracy',n_jobs=-1,verbose=1)
grid_svm.fit(X_train,Y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.1

GridSearchCV(cv=5, estimator=SVC(probability=True, random_state=21), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 1.5, 5, 10],
                         'class_weight': ['balanced', 'None'],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             scoring='accuracy', verbose=1)

In [9]:
grid_svm.best_params_

{'C': 10, 'class_weight': 'balanced', 'gamma': 'auto', 'kernel': 'rbf'}

In [10]:
grid_svm.best_estimator_

SVC(C=10, class_weight='balanced', gamma='auto', probability=True,
    random_state=21)

In [11]:
grid_svm.cv_results_

{'mean_fit_time': array([4.15078735e-01, 6.35190773e-01, 5.68764734e-01, 3.77743626e-01,
        5.74958801e-01, 5.37505245e-01, 6.50787354e-04, 5.93614578e-04,
        6.30140305e-04, 6.13594055e-04, 6.01577759e-04, 6.06727600e-04,
        1.23302097e+00, 5.69020271e-01, 5.78750086e-01, 1.17841768e+00,
        6.18405104e-01, 5.67804766e-01, 4.94575500e-04, 5.75017929e-04,
        5.52701950e-04, 6.11162186e-04, 6.16168976e-04, 4.00400162e-04,
        5.83666496e+00, 4.73578167e-01, 3.84027767e-01, 5.91737342e+00,
        4.38838148e-01, 5.39704323e-01, 6.46638870e-04, 6.65140152e-04,
        6.67333603e-04, 8.97073746e-04, 6.52217865e-04, 6.19745255e-04,
        7.43554769e+00, 5.04556417e-01, 3.62687111e-01, 7.82945805e+00,
        4.12552881e-01, 5.36339474e-01, 6.47115707e-04, 6.36243820e-04,
        6.35099411e-04, 6.83212280e-04, 6.06155396e-04, 6.20603561e-04,
        1.87234267e+01, 5.19562769e-01, 3.71871233e-01, 1.84760740e+01,
        4.25287771e-01, 5.05856657e-01, 6.36577

In [12]:
result_df_svc = pd.DataFrame(grid_svm.cv_results_)
result_df_svc.sort_values(by='rank_test_score',ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
64,0.459930,0.027861,0.025037,0.003643,10.0,balanced,auto,rbf,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.877778,0.851852,0.862963,0.873606,0.851301,0.863500,0.010870,1
52,0.425288,0.047556,0.031123,0.001591,5.0,balanced,auto,rbf,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.844444,0.785185,0.792593,0.817844,0.799257,0.807865,0.021257,2
60,30.571633,3.160461,0.007517,0.002515,10.0,balanced,scale,linear,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.729630,0.700000,0.755556,0.754647,0.665428,0.721052,0.034438,3
63,22.591699,2.163894,0.006199,0.000848,10.0,balanced,auto,linear,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.729630,0.700000,0.755556,0.754647,0.665428,0.721052,0.034438,3
48,18.723427,1.013556,0.010115,0.001495,5.0,balanced,scale,linear,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.725926,0.692593,0.696296,0.754647,0.661710,0.706234,0.031619,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42,0.000647,0.000136,0.000000,0.000000,1.5,None,scale,linear,"{'C': 1.5, 'class_weight': 'None', 'gamma': 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
43,0.000636,0.000051,0.000000,0.000000,1.5,None,scale,rbf,"{'C': 1.5, 'class_weight': 'None', 'gamma': 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
44,0.000635,0.000015,0.000000,0.000000,1.5,None,scale,sigmoid,"{'C': 1.5, 'class_weight': 'None', 'gamma': 's...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
46,0.000606,0.000156,0.000000,0.000000,1.5,None,auto,rbf,"{'C': 1.5, 'class_weight': 'None', 'gamma': 'a...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [13]:
param_grid_tree = {
    'max_depth' : range(1,50),
    'criterion' : ['entropy','gini'],
    'class_weight' : ['balanced','None']
}

In [14]:
model_tree = DecisionTreeClassifier(random_state=21)

In [15]:
grid_tree = GridSearchCV(model_tree,param_grid=param_grid_tree,n_jobs=-1,verbose=1,cv=5,scoring='accuracy')
grid_tree.fit(X_train,Y_train)


Fitting 5 folds for each of 196 candidates, totalling 980 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
490 fits failed out of a total of 980.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
490 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.1

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'None'],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 50)},
             scoring='accuracy', verbose=1)

In [16]:
grid_tree.best_estimator_

DecisionTreeClassifier(class_weight='balanced', max_depth=23, random_state=21)

In [17]:
grid_tree.best_params_

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 23}

In [18]:
result_df_tree = pd.DataFrame(grid_tree.cv_results_)
result_df_tree.sort_values(by='rank_test_score',ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
97,0.012042,0.007818,0.002998,0.001540,balanced,gini,49,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.877323,0.832714,0.873859,0.023972,1
71,0.006625,0.001855,0.002595,0.001827,balanced,gini,23,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.877323,0.832714,0.873859,0.023972,1
72,0.004613,0.001641,0.001946,0.001011,balanced,gini,24,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.877323,0.832714,0.873859,0.023972,1
73,0.005285,0.001075,0.001767,0.000329,balanced,gini,25,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.877323,0.832714,0.873859,0.023972,1
75,0.004465,0.000873,0.001180,0.000421,balanced,gini,27,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.877323,0.832714,0.873859,0.023972,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,0.000565,0.000213,0.000000,0.000000,None,entropy,28,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,99
124,0.001382,0.000702,0.000000,0.000000,None,entropy,27,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,99
123,0.000289,0.000116,0.000000,0.000000,None,entropy,26,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,99
121,0.000419,0.000253,0.000000,0.000000,None,entropy,24,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,99


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [20]:
param_grid_random = {
    'n_estimators' : [5,10,50,100],
    'max_depth' : range(1,50),
    'class_weight' : ['balanced','None'],
    'criterion' : ['entropy','gini']
}

In [21]:
model_random = RandomForestClassifier(random_state=21)

In [22]:
grid_random = GridSearchCV(model_random,param_grid_random,scoring='accuracy',n_jobs=-1,verbose=1,cv=5)
grid_random.fit(X_train,Y_train)

Fitting 5 folds for each of 784 candidates, totalling 3920 fits


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
1960 fits failed out of a total of 3920.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1360 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', 'None'],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': range(1, 50),
                         'n_estimators': [5, 10, 50, 100]},
             scoring='accuracy', verbose=1)

In [23]:
grid_random.best_estimator_

RandomForestClassifier(class_weight='balanced', max_depth=35, n_estimators=50,
                       random_state=21)

In [24]:
grid_random.best_params_

{'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 35,
 'n_estimators': 50}

In [26]:
result_df_random = pd.DataFrame(grid_random.cv_results_)
result_df_random.sort_values(by='rank_test_score',ascending=True)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
390,0.063156,7.283377e-03,0.004500,0.001603,balanced,gini,49,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.896296,0.907063,0.884758,0.903549,0.012503,1
334,0.075768,6.686122e-03,0.005093,0.001141,balanced,gini,35,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.896296,0.907063,0.884758,0.903549,0.012503,1
338,0.074618,1.300663e-02,0.003652,0.001256,balanced,gini,36,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.896296,0.907063,0.884758,0.903549,0.012503,1
342,0.063015,8.586005e-03,0.006238,0.001642,balanced,gini,37,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.896296,0.907063,0.884758,0.903549,0.012503,1
346,0.071909,7.294807e-03,0.003900,0.001072,balanced,gini,38,50,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.896296,0.907063,0.884758,0.903549,0.012503,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,0.000219,1.247091e-06,0.000000,0.000000,None,entropy,32,50,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,393
517,0.000222,2.539354e-06,0.000000,0.000000,None,entropy,32,10,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,393
516,0.000333,1.450888e-04,0.000000,0.000000,None,entropy,32,5,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,393
514,0.000218,5.001110e-07,0.000000,0.000000,None,entropy,31,50,"{'class_weight': 'None', 'criterion': 'entropy...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,393


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [39]:
param_grid_random = {
    'n_estimators': [5, 10, 50, 100],
    'max_depth': range(1, 50),
    'class_weight': ['balanced', None],
    'criterion': ['entropy', 'gini']
}

In [40]:
list_main = list()
for n_estimator in tqdm(param_grid_random['n_estimators'],desc="n_estimators"):
    for max_depth in param_grid_random['max_depth']:
        for class_weith in param_grid_random['class_weight']:
            for criterion in param_grid_random['criterion']:
                new_model = RandomForestClassifier(n_estimators=n_estimator,max_depth=max_depth,class_weight=class_weith,criterion=criterion,random_state=21,n_jobs=-1)
                accur_score = cross_val_score(new_model,X_train,Y_train,cv=5,scoring='accuracy')
                list_main.append({
                    'n_estimators' : n_estimator,
                    'max_depth' : max_depth,
                    'class_weight' : class_weith,
                    'criterion' : criterion,
                    'mean_accuracy' : accur_score.mean(),
                    'std_accuracy' : accur_score.std()
                })

n_estimators: 100%|██████████| 4/4 [03:25<00:00, 51.30s/it]


In [43]:
res_df = pd.DataFrame(list_main)

res_df.sort_values(by='mean_accuracy',ascending=False)

,n_estimators,max_depth,class_weight,criterion,mean_accuracy,std_accuracy
503,50,28,None,gini,0.904290,0.010961
711,100,31,None,gini,0.904287,0.015204
569,50,45,balanced,gini,0.903549,0.012503
561,50,43,balanced,gini,0.903549,0.012503
565,50,44,balanced,gini,0.903549,0.012503
...,...,...,...,...,...,...
2,5,1,None,entropy,0.353832,0.016467
4,5,2,balanced,entropy,0.353110,0.021165
5,5,2,balanced,gini,0.346419,0.029749
1,5,1,balanced,gini,0.283390,0.011062


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [52]:
model_best = RandomForestClassifier(class_weight=None,criterion='gini',n_estimators=50,max_depth=50,random_state=21)
model_best.fit(X_train,Y_train)
y_pr = model_best.predict(x_test)
best_acc = accuracy_score(y_test,y_pr)
best_acc

0.9319526627218935